In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import h5py

class SequenceDataset(Dataset):
    def __init__(self, h5_file):
        self.h5_file = h5py.File(h5_file, "r")
        self.images = self.h5_file["images"]  # Shape: (N, 32, 32)
        self.labels = self.h5_file["labels"]  # Shape: (N,)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Get image and reshape to a sequence of 1024 pixels
        image = self.images[idx]  # Shape: (32, 32)
        image = image.flatten()  # Shape: (1024,)
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(-1)  # Shape: (1024, 1)
        
        # Get label
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return image, label


In [6]:
# Hyperparameters
vocab_size = 256  # Pixel values range from 0 to 255
embedding_dim = 64
batch_size = 32
learning_rate = 0.001
epochs = 10
test_size = 0.2  # Fraction of the dataset to use for testing

# Load the dataset
full_dataset = SequenceDataset("/kaggle/input/easydata/merged_data (3).h5")

# Calculate the sizes of the train and test sets
train_size = int((1 - test_size) * len(full_dataset))
test_size = len(full_dataset) - train_size

# Split the dataset
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
test_size2 = 0.2
full_dataset2 = SequenceDataset("/kaggle/input/mediumdata/merged_data (2).h5")

# Calculate the sizes of the train and test sets
train_size2 = int((1 - test_size2) * len(full_dataset2))
test_size2 = len(full_dataset2) - train_size2

# Split the dataset
train_dataset2, test_dataset2 = random_split(full_dataset2, [train_size2, test_size2])

# Create data loaders
train_dataloader2 = DataLoader(train_dataset2, batch_size=batch_size, shuffle=True)
test_dataloader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False)


In [7]:
full_dataset3= SequenceDataset("/kaggle/input/harddata/merged_data (1).h5")
test_size3 = 0.2
# Calculate the sizes of the train and test sets
train_size3 = int((1 - test_size3) * len(full_dataset3))
test_size3 = len(full_dataset3) - train_size3

# Split the dataset
train_dataset3, test_dataset3 = random_split(full_dataset3, [train_size3, test_size3])

# Create data loaders
train_dataloader3 = DataLoader(train_dataset3, batch_size=batch_size, shuffle=True)
test_dataloader3= DataLoader(test_dataset3, batch_size=batch_size, shuffle=False)


In [ ]:
!pip install performer-pytorch


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from performer_pytorch import Performer
from tqdm import tqdm  # Import tqdm for progress bars

class PerformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes=2, dim_head=64):
        super(PerformerModel, self).__init__()
        
        # Embedding layer for pixel values
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Performer Model
        self.performer = Performer(
            dim=embedding_dim,
            depth=4,  # Number of layers
            heads=8,  # Number of attention heads
            dim_head=dim_head,  # Size of each attention head
            kernel_fn="relu",  # You can use 'relu' or 'cosine'
            causal=False
        )
        
        # Final classification layer (after performer output)
        self.fc = nn.Linear(embedding_dim, num_classes)
        
    def forward(self, x):
        # x has shape (batch_size, sequence_length, 1)
        x = x.squeeze(-1)  # (batch_size, sequence_length)
        
        # Cast the input to long (integer) type for the embedding layer
        x = x.long()  # Convert to long (integer) tensor
        
        x = self.embedding(x)  # (batch_size, sequence_length, embedding_dim)
        
        # Pass through performer
        x = self.performer(x)  # (batch_size, sequence_length, embedding_dim)
        
        # Take the output of the last token (or perform pooling) for classification
        x = x.mean(dim=1)  # Mean pooling over the sequence
        
        # Pass through fully connected layer
        x = self.fc(x)  # (batch_size, num_classes)
        
        return x

# Set device to GPU (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PerformerModel(vocab_size=256, embedding_dim=64).to(device)

# Initialize loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Binary classification (2 classes)
optimizer = optim.Adam(model.parameters(), lr=1e-5)  # Try even lower learning rate

# Set up mixed precision training
scaler = torch.cuda.amp.GradScaler()




In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    # Using tqdm for the progress bar in the training loop
    with tqdm(train_dataloader1, unit="batch", ncols=100, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)  # Move to device
            
            # Check for NaN values in images and labels
            if torch.any(torch.isnan(images)) or torch.any(torch.isnan(labels)):
                print("NaN detected in input data!")
                continue
            
            # Forward pass with mixed precision
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():  # Enable mixed precision
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            # Check if loss is NaN
            if torch.isnan(loss):
                print(f"NaN detected in loss at epoch {epoch+1}, batch {pbar.n}")
                continue
            
            # Backward pass and optimization with scaled gradients
            scaler.scale(loss).backward()

            # Gradient clipping (to prevent exploding gradients)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update gradients with optimizer step
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            
            # Update the progress bar with the current loss
            pbar.set_postfix(loss=running_loss / (pbar.n + 1), refresh=True)
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_dataloader)}")

# Test the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader1:
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    # Using tqdm for the progress bar in the training loop
    with tqdm(train_dataloader2, unit="batch", ncols=100, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)  # Move to device
            
            # Check for NaN values in images and labels
            if torch.any(torch.isnan(images)) or torch.any(torch.isnan(labels)):
                print("NaN detected in input data!")
                continue
            
            # Forward pass with mixed precision
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():  # Enable mixed precision
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            # Check if loss is NaN
            if torch.isnan(loss):
                print(f"NaN detected in loss at epoch {epoch+1}, batch {pbar.n}")
                continue
            
            # Backward pass and optimization with scaled gradients
            scaler.scale(loss).backward()

            # Gradient clipping (to prevent exploding gradients)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update gradients with optimizer step
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            
            # Update the progress bar with the current loss
            pbar.set_postfix(loss=running_loss / (pbar.n + 1), refresh=True)
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_dataloader)}")

# Test the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader2:
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    # Using tqdm for the progress bar in the training loop
    with tqdm(train_dataloader3, unit="batch", ncols=100, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)  # Move to device
            
            # Check for NaN values in images and labels
            if torch.any(torch.isnan(images)) or torch.any(torch.isnan(labels)):
                print("NaN detected in input data!")
                continue
            
            # Forward pass with mixed precision
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():  # Enable mixed precision
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            # Check if loss is NaN
            if torch.isnan(loss):
                print(f"NaN detected in loss at epoch {epoch+1}, batch {pbar.n}")
                continue
            
            # Backward pass and optimization with scaled gradients
            scaler.scale(loss).backward()

            # Gradient clipping (to prevent exploding gradients)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update gradients with optimizer step
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            
            # Update the progress bar with the current loss
            pbar.set_postfix(loss=running_loss / (pbar.n + 1), refresh=True)
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_dataloader)}")

# Test the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader3:
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")
